# Set up

In [ ]:
!pip3 install auto-sklearn

In [ ]:
import numpy as np; np.random.seed(42)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from autosklearn.regression import AutoSklearnRegressor
from autosklearn.classification import AutoSklearnClassifier
import autosklearn
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir 'Data'
!unzip 'drive/MyDrive/Kaggle/electric-power-consumption.zip' -d 'Data'

Mounted at /content/drive
Archive:  drive/MyDrive/Kaggle/electric-power-consumption.zip
  inflating: Data/PowerCons_TEST.csv  
  inflating: Data/PowerCons_TRAIN.csv  
  inflating: Data/README.md          
  inflating: Data/forecast_submission_example.csv  


# Prepare data and predict

In [ ]:
train = pd.read_csv('/content/Data/PowerCons_TRAIN.csv')
test = pd.read_csv('/content/Data/PowerCons_TEST.csv')

ตอนนี้พยายามแปลงข้อมูล train set ให้เหมือน test set 

In [ ]:
half_train=train.iloc[:,1:73]
another_half_train=train.iloc[:,73:len(train.columns)-2]
half_train['kW-12:00-23:50']=another_half_train.sum(axis=1)
half_train['Class']=train['Class']
half_train

,00:00,00:10,00:20,00:30,00:40,00:50,01:00,01:10,01:20,01:30,01:40,01:50,02:00,02:10,02:20,02:30,02:40,02:50,03:00,03:10,03:20,03:30,03:40,03:50,04:00,04:10,04:20,04:30,04:40,04:50,05:00,05:10,05:20,05:30,05:40,05:50,06:00,06:10,06:20,06:30,06:40,06:50,07:00,07:10,07:20,07:30,07:40,07:50,08:00,08:10,08:20,08:30,08:40,08:50,09:00,09:10,09:20,09:30,09:40,09:50,10:00,10:10,10:20,10:30,10:40,10:50,11:00,11:10,11:20,11:30,11:40,11:50,kW-12:00-23:50,Class
0,0.44,0.40,0.36,0.35,0.35,0.38,0.36,0.35,0.35,0.38,0.38,0.38,0.42,0.45,0.43,0.37,0.66,1.13,1.35,1.09,0.75,0.41,0.37,0.33,0.33,0.40,0.40,0.39,0.40,0.43,0.40,0.35,0.31,0.33,0.33,0.33,0.30,0.30,0.33,0.36,1.10,1.83,1.93,2.04,2.04,1.93,1.93,1.53,1.36,1.19,2.25,3.31,3.08,2.85,2.04,1.24,1.26,1.29,1.31,1.34,1.79,2.25,1.73,1.21,1.12,0.97,0.82,0.47,0.11,0.16,0.22,0.62,41.30,1
1,0.14,0.19,0.19,0.16,0.16,0.18,0.17,0.16,0.17,0.17,0.16,0.18,0.18,0.14,0.13,0.16,0.19,0.14,0.15,0.18,0.18,0.15,0.16,0.19,0.17,0.16,0.16,0.17,0.16,0.18,0.17,0.14,0.15,0.17,0.18,0.10,0.37,0.64,0.89,1.14,1.19,1.24,0.68,0.12,0.16,0.17,0.16,0.18,0.16,0.14,0.15,0.16,0.16,0.13,0.16,0.14,0.12,0.14,0.17,0.18,0.16,0.16,0.16,0.16,0.17,0.22,0.25,0.25,0.27,0.29,0.17,0.13,27.79,1
2,0.34,0.34,0.34,0.26,0.18,0.83,1.48,1.36,0.95,0.55,0.42,0.29,0.26,0.24,0.24,0.25,0.23,0.25,0.26,0.25,0.25,0.26,0.28,0.30,0.33,0.31,0.30,0.27,0.25,0.22,0.20,0.22,0.24,0.22,0.19,0.22,0.24,0.24,0.25,1.79,3.33,2.94,2.54,2.54,1.58,1.58,1.46,1.34,1.61,1.79,1.96,1.97,1.63,1.29,1.31,1.34,1.32,1.30,1.28,1.26,1.24,1.21,0.92,0.64,0.45,0.26,0.23,0.24,0.26,0.25,0.25,0.29,51.04,1
3,1.70,1.50,1.30,3.40,5.50,4.15,2.80,2.67,2.53,2.49,2.45,2.41,1.64,0.88,0.53,0.19,0.17,0.16,0.16,0.22,0.29,0.19,0.14,0.15,0.21,0.26,0.26,0.22,0.21,0.24,0.26,0.29,0.22,0.17,0.13,0.19,0.24,0.16,0.13,0.17,0.21,0.26,0.30,0.99,1.67,1.52,1.36,1.38,1.38,1.39,1.41,1.38,1.35,1.34,1.33,1.33,1.31,1.30,0.89,0.49,0.31,0.14,0.20,0.23,0.22,0.23,0.28,0.31,0.30,0.28,0.34,0.41,67.77,1
4,1.78,1.59,1.39,1.45,1.45,1.41,1.40,1.45,1.46,1.40,0.85,0.30,0.34,0.38,0.33,0.33,0.39,0.35,0.31,0.31,0.35,0.38,0.38,0.33,0.33,0.38,0.39,0.35,0.33,0.36,0.38,0.37,0.34,0.34,0.37,0.37,0.35,0.35,0.37,0.38,0.36,0.35,0.36,0.36,0.36,0.35,0.37,0.37,0.35,0.34,0.36,0.38,0.37,0.39,0.41,1.08,1.75,1.68,1.66,1.65,1.50,1.34,1.55,1.75,1.68,1.60,1.48,1.41,1.38,1.33,1.31,1.34,31.31,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.39,0.38,0.34,0.29,0.27,0.26,0.24,0.25,0.26,0.82,1.37,1.30,1.16,1.03,0.66,0.29,0.28,0.27,0.27,0.27,0.28,0.30,0.31,0.31,0.28,0.25,0.28,0.31,0.35,0.35,0.30,0.30,0.31,0.27,0.24,0.29,0.28,0.27,0.28,0.30,1.23,2.16,2.08,2.00,2.01,2.01,2.06,2.11,2.18,2.24,2.55,2.86,2.71,2.56,2.38,2.19,2.11,2.02,1.81,1.71,1.61,1.57,1.54,2.61,3.68,3.73,3.27,2.81,2.22,1.63,1.66,1.69,121.96,2
176,0.71,0.49,0.27,0.31,0.36,0.43,0.42,0.40,0.31,0.27,0.22,0.24,0.26,0.27,0.30,0.34,0.30,0.25,0.28,0.27,0.25,0.25,0.27,0.25,0.23,0.23,0.71,1.20,1.26,1.31,1.12,0.68,0.25,0.27,0.29,0.24,0.23,0.30,0.38,0.29,1.42,2.55,2.58,2.38,2.19,2.43,2.68,2.23,1.78,2.20,2.61,1.51,0.41,0.36,0.30,0.41,0.75,1.09,0.77,0.44,0.46,0.48,0.42,0.35,0.37,0.39,0.47,0.52,0.56,0.71,0.85,0.77,114.52,2
177,0.39,0.31,0.26,0.30,0.39,0.38,0.31,0.26,0.29,0.32,0.30,0.26,0.27,0.31,0.32,0.28,0.29,0.35,0.38,0.33,0.28,0.28,0.29,0.29,0.27,0.28,0.29,0.30,0.27,0.30,0.35,0.36,0.31,0.27,0.28,0.32,0.69,1.06,1.40,1.75,1.90,2.06,4.93,7.80,5.55,3.30,2.70,2.10,2.19,2.29,1.85,1.41,1.43,1.41,2.02,2.64,2.01,1.38,1.36,1.39,1.43,1.41,1.36,1.32,1.36,1.39,1.37,1.33,1.34,1.39,1.40,1.38,176.74,2
178,0.41,0.34,0.29,0.23,0.27,0.32,0.25,0.24,0.28,0.32,0.32,0.29,0.33,0.36,0.31,0.26,0.24,0.27,0.27,0.27,0.28,0.31,0.33,0.31,0.26,0.23,0.24,0.25,0.27,0.25,0.25,0.31,0.37,0.30,0.22,0.23,0.27,0.31,0.25,0.25,0.29,1.96,3.63,2.88,2.13,2.19,1.95,1.71,1.71,1.71,1.60,1.50,1.05,0.60,0.5

สร้าง model สำหรับทำนาย class เนื่องจากข้อมูล test set ไม่มี class

In [ ]:
classification_model=AutoSklearnClassifier(time_left_for_this_task=60, resampling_strategy_arguments={'cv':5}, metric=autosklearn.metrics.accuracy, seed=42)

Time_data=half_train.drop(['kW-12:00-23:50','Class'],axis=1)
Class=half_train['Class']

Time_data_train,Time_data_validate,Class_train,Class_validate=train_test_split(Time_data,Class,test_size=0.2)

classification_model.fit(Time_data_train,Class_train)

predict=classification_model.predict(Time_data_validate)
print('Accuracy of this model : {}'.format(accuracy_score(predict,Class_validate)))
print(classification_model.leaderboard())

Accuracy of this model : 0.75
          rank  ensemble_weight               type      cost  duration
model_id                                                              
11           1             0.12      random_forest  0.229167  2.040986
3            2             0.16  gradient_boosting  0.250000  1.988381
6            3             0.08      random_forest  0.250000  2.339293
9            4             0.12      random_forest  0.250000  3.792956
15           5             0.02  gradient_boosting  0.250000  1.763016
14           6             0.10      random_forest  0.270833  2.409349
18           7             0.04         libsvm_svc  0.270833  0.979775
20           8             0.06           adaboost  0.270833  2.274085
2            9             0.14      random_forest  0.291667  1.969742
7           10             0.06      random_forest  0.291667  2.381091
17          11             0.02  gradient_boosting  0.312500  1.467611
19          12             0.04                

เนื่องจากข้อมูลมีความ multicolinear สูงจึงทำการหา difference เผื่อมันจะช่วยแก้ปัญหานี้ได้ก่อนที่จะนำไปเทรน regression

In [ ]:
X=half_train.drop(['kW-12:00-23:50'],axis=1)
y=half_train['kW-12:00-23:50']

Xtrain,Xvalidate,ytrain,yvalidate=train_test_split(X,y,test_size=0.2)
Xtrain.index=range(len(Xtrain.index))
ytrain.index=range(len(Xtrain.index))
Xvalidate.index=range(len(Xvalidate.index))
yvalidate.index=range(len(Xvalidate.index))

#หา Class ด้วย เพื่อทดสอบว่า pipeline นี้มีความแม่นยำประมาณเท่าไร (ยกเลิก)
#Xtrain_Class=pd.Series(classification_model.predict(Xtrain))

diff_train=pd.DataFrame()

first=list(Xtrain.columns)[:-2]
second=list(Xtrain.columns)[1:-1]
for t1,t2 in zip(first,second):
  diff_train=pd.concat([diff_train,Xtrain[t2]-Xtrain[t1]],axis=1)
diff_train.columns=['t'+str(i) for i in range(len(Xtrain.columns)-2)]
diff_train=pd.concat([diff_train,Xtrain['Class']],axis=1)
#diff_train.rename(columns={0:'Class'},inplace=True)

diff_train

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21,t22,t23,t24,t25,t26,t27,t28,t29,t30,t31,t32,t33,t34,t35,t36,t37,t38,t39,t40,t41,t42,t43,t44,t45,t46,t47,t48,t49,t50,t51,t52,t53,t54,t55,t56,t57,t58,t59,t60,t61,t62,t63,t64,t65,t66,t67,t68,t69,t70,Class
0,0.31,0.31,0.30,0.31,-0.01,0.00,0.05,-0.01,-0.07,-0.03,0.02,0.04,-0.01,0.01,0.01,0.01,-0.03,0.03,0.03,0.03,-0.05,-0.02,-0.01,0.01,-0.01,0.01,0.01,0.00,-0.03,0.01,-0.01,0.02,0.00,0.03,-0.01,-0.03,-0.02,0.02,0.00,-0.03,-0.01,0.01,0.00,-0.05,0.00,0.04,0.03,-0.03,-0.02,0.02,-0.01,-0.04,0.00,0.03,0.00,-0.05,0.01,0.11,0.20,0.37,0.38,-0.28,-0.29,0.00,-2.28,0.00,-0.16,-0.06,0.03,0.06,-0.01,2
1,0.21,0.20,0.63,0.63,-0.26,-0.25,-0.62,-0.63,0.09,0.00,-0.05,-0.01,0.04,0.01,-0.02,-0.01,0.01,0.00,0.00,0.02,-0.02,-0.03,-0.02,0.02,0.03,-0.03,0.02,0.03,-0.01,-0.01,0.00,0.00,-0.01,0.03,0.00,-0.03,0.00,0.04,-0.01,-0.03,-0.01,0.03,0.00,-0.01,0.01,-0.01,-0.02,0.18,0.42,0.45,0.82,0.82,-0.27,-0.27,-0.43,-0.42,0.28,0.29,0.81,0.81,-0.90,-0.90,0.29,0.29,0.29,0.29,-0.41,-0.73,-0.74,0.06,0.02,1
2,-0.09,-0.01,0.03,0.01,-0.06,-0.01,0.03,-0.01,-0.02,-0.01,0.02,0.03,0.01,-0.02,0.02,0.05,-0.01,-0.06,-0.03,0.01,0.01,-0.01,-0.02,0.02,0.00,-0.01,-0.01,0.02,0.00,-0.01,-0.01,0.05,0.05,-0.01,-0.06,-0.02,0.04,0.04,1.09,1.08,-0.11,-0.11,-0.02,-0.01,0.08,-0.11,-0.10,-0.06,-0.31,-0.31,-0.04,-0.06,-0.04,0.00,0.02,-0.01,0.00,0.01,-0.01,0.00,0.00,0.03,0.00,-0.03,0.04,0.05,0.02,-0.05,-0.06,-0.03,0.04,1
3,-0.01,0.01,0.01,-0.02,-0.03,0.19,0.20,-0.02,-0.01,0.00,0.01,0.00,-0.02,-0.02,-0.01,-0.01,-0.12,-0.13,-0.03,0.01,0.01,0.00,0.04,0.00,0.60,0.61,0.00,-1.24,0.00,-0.03,-0.01,-0.02,0.02,0.02,0.03,-0.04,-0.03,0.32,0.33,-0.19,-0.18,0.01,0.01,0.09,0.09,0.53,0.54,1.01,0.00,-1.09,0.00,0.14,0.01,0.02,-0.16,-0.17,0.11,0.11,-0.01,0.01,0.02,-0.03,0.07,0.07,-0.09,-0.20,-0.20,-0.04,0.00,0.01,0.04,2
4,-0.04,-0.06,-0.05,0.03,0.03,-0.01,-0.01,0.03,0.05,0.00,-0.03,-0.03,-0.01,-0.01,0.00,-0.02,0.01,0.01,0.01,-0.03,0.00,0.03,0.04,0.00,0.00,0.03,-0.01,-0.06,-0.04,0.02,0.02,-0.02,-0.03,0.02,0.04,0.00,0.59,0.60,0.37,0.37,-0.12,-0.14,-0.14,0.06,0.00,-0.13,-0.21,-0.18,-0.09,-0.02,0.04,0.02,-0.02,-0.01,0.05,0.04,-0.01,-0.37,-0.38,-0.18,-0.15,-0.01,0.02,0.03,-0.03,-0.06,0.00,0.05,0.01,-0.03,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,-0.20,-0.19,-0.39,-0.38,-0.05,-0.05,0.04,0.03,0.02,0.01,0.00,-0.01,-0.02,-0.02,0.02,-0.01,-0.02,0.00,0.00,0.00,0.00,0.00,0.03,0.03,0.01,-0.02,0.01,0.00,-0.04,-0.05,0.03,0.05,-0.01,-0.01,0.00,0.00,-0.04,-0.03,0.02,0.06,0.10,0.38,0.39,-0.04,-0.04,0.40,0.40,0.32,-0.26,-0.26,-0.12,0.04,0.04,-0.17,-0.17,0.02,0.03,0.00,0.00,-0.01,0.00,-0.01,-0.03,-0.04,0.09,0.09,-0.04,-0.05,1.27,1.27,-0.02,2
140,-0.18,-0.18,-0.07,0.00,-0.29,0.00,0.00,-2.48,0.00,-0.01,0.02,0.02,0.36,0.36,0.00,-0.87,0.00,0.00,0.06,-0.01,0.02,0.03,-0.02,-0.01,-0.04,-0.03,0.05,0.04,-0.05,-0.04,0.06,0.00,0.00,-0.03,-0.03,0.02,0.03,0.00,0.03,0.00,-0.03,-0.03,0.01,0.01,0.00,0.00,-0.04,0.27,0.27,0.27,0.27,0.00,-0.62,0.00,-0.17,-0.18,0.05,0.07,0.07,0.49,0.50,0.46,-0.09,-0.10,0.00,-0.07,0.00,0.11,0.11,0.45,0.45,2
141,0.03,-0.01,-0.02,0.01,0.02,-0.01,-0.03,0.03,0.01,-0.04,0.38,0.38,0.26,-0.32,-0.32,-0.21,-0.21,0.08,0.00,-0.04,0.01,0.03,-0.01,-0.01,0.00,0.01,0.00,0.01,-0.02,-0.01,0.03,-0.01,-0.03,0.00,0.03,0.01,-0.02,0.00,0.00,0.01,0.06,0.23,0.23,0.38,0.38,0.57,0.57,-0.38,-0.42,-0.33,-0.32,0.02,0.00,0.01,0.02,-0.02,-0.02,0.04,-0.01,-0.02,-0.01,0.03,0.00,0.00,0.00,-0.02,-0.03,-0.27,-0.27,-0.22,-0.22,1
142,0.12,0.02,-0.07,-0.37,-0.37,-0.19,-0.14,-0.04,0.00,0.03,0.01,0.00,-0.02,0.00,0.05,0.04,0.00,-0.09,-0.01,0.04,0.00,-0.04,-0.02,0.03,0.03,-0.01,-0.02,-0.01,0.06,0.05,-0.01,-0.06,-0.05,0.01,0.02,0.00,-0.05,0.02,0.10,0.86,0.86,0.06,0.06,0.11,0.17,0.20,0.19,-0.66,-0.67,-0.09,-0.07,0.24,0.24,0.04,-0

In [ ]:
def rmse(y,predict):
  return np.sqrt((np.sum((y-predict)**2))/len(y))

ทดสอบประสิทธิภาพ regression (หากมันได้รับ class ที่ถูกต้อง)

In [ ]:
#Filter column (Comment to stop filter)
#diff_train=diff_train.loc[:,['t0','t3','t10','t11','t37','t45','t47','t49','t51','t53','t68','t70','Class']]

regression_model=AutoSklearnRegressor(time_left_for_this_task=60, resampling_strategy_arguments={'cv':5}, metric=autosklearn.metrics.mean_squared_error, seed=42)

regression_model.fit(diff_train,ytrain)

#Xvalidate_Class=pd.Series(classification_model.predict(Xvalidate))

diff_validate=pd.DataFrame()

first=list(Xvalidate.columns)[:-2]
second=list(Xvalidate.columns)[1:-1]
for t1,t2 in zip(first,second):
  diff_validate=pd.concat([diff_validate,Xvalidate[t2]-Xvalidate[t1]],axis=1)
diff_validate.columns=['t'+str(i) for i in range(len(Xvalidate.columns)-2)]
diff_validate=pd.concat([diff_validate,Xvalidate['Class']],axis=1)
#diff_validate.rename(columns={0:'Class'},inplace=True)

#Filter column (Comment to stop filter)
#diff_validate=diff_validate.loc[:,['t0','t3','t10','t11','t37','t45','t47','t49','t51','t53','t68','t70','Class']] 

predict=regression_model.predict(diff_validate)
loss=rmse(yvalidate,predict)

print('RMSE : {}'.format(loss))
print(regression_model.leaderboard())

RMSE : 25.76069746721829
          rank  ensemble_weight               type         cost  duration
model_id                                                                 
2            1             0.56      random_forest  1076.390141  2.406964
16           2             0.32        extra_trees  1095.442988  1.676461
13           3             0.08  gradient_boosting  1158.576467  2.111997
4            4             0.04   gaussian_process  2016.471293  1.162117


In [ ]:
#nn_model = Sequential()
#nn_model.add(Dense(128, input_dim=df_train.shape[1], kernel_initializer='normal', activation='relu'))
#nn_model.add(Dropout(0.3))
#nn_model.add(Dense(64, kernel_initializer='normal', activation='relu'))
#nn_model.add(Dropout(0.3))
#nn_model.add(Dense(64, kernel_initializer='normal', activation='relu'))
#nn_model.add(Dropout(0.3))
#nn_model.add(Dense(1, kernel_initializer='normal'))

#nn_model.compile(loss='mean_squared_error', optimizer='adam',metrics=[tf.keras.metrics.RootMeanSquaredError()])

#history = nn_model.fit(x=df_train, y=ytrain, batch_size=36, epochs=300, verbose=1)

In [ ]:
test_nodate=test.drop('Day',axis=1)
test_Class=pd.Series(classification_model.predict(test_nodate))

diff_test=pd.DataFrame()

first=list(test_nodate.columns)[:-1]
second=list(test_nodate.columns)[1:]
for t1,t2 in zip(first,second):
  diff_test=pd.concat([diff_test,test_nodate[t2]-test_nodate[t1]],axis=1)
diff_test.columns=['t'+str(i) for i in range(len(test_nodate.columns)-1)]
diff_test=pd.concat([diff_test,test_Class],axis=1)
diff_test.rename(columns={0:'Class'},inplace=True)
diff_test.to_csv('Waiting for pycaret.csv')
diff_test

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21,t22,t23,t24,t25,t26,t27,t28,t29,t30,t31,t32,t33,t34,t35,t36,t37,t38,t39,t40,t41,t42,t43,t44,t45,t46,t47,t48,t49,t50,t51,t52,t53,t54,t55,t56,t57,t58,t59,t60,t61,t62,t63,t64,t65,t66,t67,t68,t69,t70,Class
0,-0.03,-0.01,-0.01,0.00,0.01,0.01,0.49,0.48,0.06,0.06,-0.57,-0.56,0.00,0.06,-0.01,-0.10,0.09,0.00,-0.01,-0.01,-0.01,-0.02,0.01,0.01,-0.06,0.08,0.07,-0.07,-0.08,0.03,0.03,-0.02,-0.02,0.01,0.01,0.03,0.02,-0.03,-0.04,0.78,0.78,-0.33,0.27,0.27,0.38,0.37,0.00,-1.23,0.00,-0.14,-0.03,-0.04,0.02,-0.03,-0.03,-0.01,0.00,0.04,0.03,-0.01,-0.01,-0.50,-0.50,0.00,-0.05,0.00,0.00,0.11,0.00,0.74,0.74,2
1,0.00,0.00,0.00,0.04,0.00,-0.04,-0.03,0.02,0.01,0.02,-0.01,-0.01,0.00,0.03,0.00,-0.04,-0.05,0.00,0.01,0.02,0.03,-0.03,0.02,0.02,-0.05,-0.05,0.00,0.00,0.06,0.06,-0.07,-0.07,0.00,0.06,0.00,0.02,0.02,0.00,0.00,1.20,1.20,-0.53,-0.52,0.02,0.02,-0.42,-0.42,0.65,0.66,-0.25,-0.25,0.09,0.01,0.02,-0.28,-0.10,-0.09,0.09,-0.05,-0.06,0.00,0.01,0.00,0.00,-0.21,-0.36,-0.35,0.00,-0.06,0.00,0.58,1
2,-0.28,-0.28,-0.64,-0.65,-0.05,0.00,0.01,0.00,-0.03,-0.02,0.02,0.01,0.00,-0.01,0.00,0.03,0.00,-0.04,-0.04,0.01,0.01,0.02,0.02,0.00,0.10,0.00,-0.07,-0.08,0.00,0.00,0.04,0.04,0.00,-0.03,0.00,0.02,0.01,0.26,0.26,0.77,0.53,0.52,0.02,0.02,0.04,0.03,0.02,0.08,0.09,-0.62,-0.63,0.41,-0.19,-0.19,-0.22,-0.23,0.09,0.03,0.03,0.00,-0.07,0.00,-0.01,-0.02,-0.04,0.00,0.11,0.00,0.08,0.09,0.32,2
3,-0.43,-0.36,-0.37,-0.65,-0.11,-0.11,-0.16,-0.28,-0.28,-0.38,-0.19,-0.20,0.02,0.01,0.00,0.01,0.02,0.00,0.00,0.00,0.01,0.01,0.02,-0.04,-0.03,0.02,0.01,-0.04,-0.04,0.01,0.03,0.03,-0.01,-0.01,0.03,-0.01,-0.01,-0.01,-0.01,0.02,0.01,-0.03,-0.04,0.23,0.22,0.28,0.29,0.00,0.06,0.00,0.00,0.85,0.85,-1.30,-1.30,1.00,0.99,-0.91,-0.92,0.42,0.43,-0.02,-0.01,1.28,1.28,-0.83,-0.83,0.50,0.49,-0.06,-0.05,2
4,-0.11,-0.10,-0.05,0.01,0.04,0.03,-0.01,-0.01,-0.02,-0.01,-0.01,-0.01,-0.04,0.55,0.56,-0.06,-0.41,-0.41,-0.13,-0.07,0.01,0.03,-0.02,-0.02,0.02,0.01,-0.01,0.01,-0.02,0.00,0.03,0.00,-0.05,0.01,0.04,-0.01,-0.03,0.01,0.02,0.00,0.48,0.48,-0.37,-0.37,0.23,0.22,0.23,0.22,-0.11,-0.10,-0.24,-0.24,-0.25,-0.24,0.03,0.01,0.00,0.01,-0.03,0.00,0.03,0.00,-0.04,0.01,0.03,0.00,-0.02,0.00,0.01,0.00,0.01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.00,-0.01,-0.03,-0.01,0.00,-0.03,-0.01,0.04,0.04,-0.02,0.01,0.01,-0.03,-0.04,0.02,0.02,0.03,-0.01,-0.01,-0.01,-0.02,0.00,0.00,0.01,0.03,0.03,-0.03,-0.03,-0.01,0.00,0.01,0.00,-0.03,-0.02,0.06,0.06,-0.06,-0.06,0.30,0.29,0.25,0.25,0.06,0.07,-0.59,-0.59,-0.01,0.00,0.04,0.03,-0.01,0.00,0.02,-0.05,-0.06,0.04,0.04,0.03,-0.01,-0.02,0.01,-0.01,-0.02,-0.02,-0.01,0.03,0.04,-0.02,-0.01,0.01,0.02,1
176,-0.09,-0.08,-0.26,-0.27,-0.26,-0.26,0.02,0.02,-0.02,-0.04,-0.02,0.03,0.00,-0.01,0.07,0.01,0.02,-0.05,-0.05,-0.01,0.03,0.01,0.01,-0.02,-0.04,0.03,0.04,-0.05,0.00,0.04,0.03,-0.02,-0.03,0.00,0.02,-0.01,-0.03,0.00,0.05,0.02,-0.01,-0.02,-0.02,0.03,-0.02,-0.02,0.00,0.00,0.01,0.00,-0.02,0.01,0.02,0.00,-0.02,0.00,0.01,-0.01,-0.02,0.01,0.01,0.01,0.01,-0.04,0.56,0.55,-0.03,-0.03,0.03,-0.24,-0.24,1
177,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,-0.02,-0.01,-0.02,-0.01,0.01,0.01,-0.02,-0.01,0.02,0.03,0.01,0.00,0.00,-0.01,-0.02,-0.01,0.02,0.03,-0.02,-0.02,-0.04,0.00,1.22,0.00,-0.10,-0.43,-0.43,-0.13,-0.12,0.04,0.03,0.07,0.34,0.34,-0.11,-0.11,0.40,0.40,0.21,0.21,-0.36,-0.35,0.01,0.15,0.15,0.07,0.06,0.03,0.03,0.00,0.00,0.01,-0.23,-0.22,-0.01,-0.01,-0.06,0.07,0.07,0.00,-0.01,-0.04,-0.04,2
178,0.11,-0.04,-0.04,0.00,0.04,-0.03,-0.03,0.01,0.04,0.00,0.00,0.01,-0.01,-0.01,0.00,-0.02,0.02,0.02,-0.01,-0.02,0.03,0.01,0.00,-0.05,0.00,0.55,0.55,-0.17,-0.18,-0.33,-0.33,-0.04,0.00,0.00,0.00,-0.04,0.01,0.02,0.01,1.54,1.54,-0.26,-0.27,-0.10,0.27,0.27,-0.39,-0.40,-0.44,-0.45,-0.01,0.00,-0.03,-0.03,0.00,-

In [ ]:
#diff_test=diff_test.loc[:,['t0','t3','t10','t11','t37','t45','t47','t49','t51','t53','t68','t70','Class']] 

predict=regression_model.predict(diff_test)
submit=pd.DataFrame({'Day':range(1,181),'kW-12:00-23:50':predict}).set_index('Day')
submit.to_csv('Submission.csv')